# Семинар 5: Потенциал AlphaFold

# Попробуем сделать предсказание структуры белка с помощью минимизации потенциала AlphaFold из имеющихся дистограмм.

# Сегодня будем работать с отсносительно короткими белками - длиной 40-50.

# 0. Приготовления!

В этом семинаре потребуется много всего. 🙂

Для начала библиотеки, с которыми мы уже работали:

In [11]:
!pip install Bio

In [12]:
import Bio.PDB as pdb
from Bio.PDB.QCPSuperimposer import QCPSuperimposer
import numpy as np
import matplotlib.pyplot as plt
import os

Потенциал требует минимизации. Оптимизировать будем с помощью модуля **scipy.optimize**:

In [13]:
from scipy.optimize import minimize

Главный герой сегодняшнего семинара. Загрузите <a href="https://disk.yandex.ru/d/3eFkKsx7coUqXA">Python модуль для вычисления потенциала AlphaFold</a> и положите его в ту же директорию, где лежит эта тетрадка.

In [14]:
from alphafold_potential.potential import Potential
from alphafold_potential.supplementary import read_pickle

# 1. Прочитаем данные:

Загрузите папку с результатами первого этапа работы AlphaFold для своего белка <a href="https://disk.yandex.ru/d/vW79A2J3-M_-gg">отсюда</a>. Положите папку целиком в ту же директорию, где находится эта тетрадка.

In [21]:
dir_name = 'contacts_pdb1h1j_2020_07_22_12_10_37'

In [22]:
pdb_id = dir_name.split("_")[1][3:]

In [23]:
p1 = f"./{dir_name}/pasted/{pdb_id}.pickle"
p2 = f"./{dir_name}/background_distogram/ensemble/{pdb_id}.pickle"
p3 = f"./{dir_name}/torsion/ensemble/{pdb_id}.pickle"
p4 = f"./{dir_name}/torsion/0/torsions/{pdb_id}.torsions"

Загрузим белок, которому соответствуют загруженные выше данные:

In [24]:
path = pdb.PDBList().retrieve_pdb_file(pdb_id, file_format="pdb")

In [26]:
parser = pdb.PDBParser()
chain = parser.get_structure("s", path)[0].child_list[0].child_list

## 1.1. Для сравнения результата минимизации с оргинальными значениями $\phi, \psi$ посчитаем все двугранные углы реального белка.

Зададим вектор **orig_angles**, в котором последовательно запишем $\phi_1, \psi_1, \phi_2, \psi_2, ..., \phi_n, \psi_n$:

**Замечание**: У первого остатка нет угла $\phi_1$, у последнего - $\psi_n$. Пусть они будут равны $\pi$.

In [32]:
orig_angles = [np.pi]
for i in range(len(chain)):
    n = chain[i]["N"].get_vector()
    ca = chain[i]["CA"].get_vector()
    c = chain[i]["C"].get_vector()
    
    if i == 0:
        phi = np.pi
    else:
        c_prev = chain[i - 1]["C"].get_vector()
        phi = pdb.calc_dihedral(c_prev, n, ca, c)
        orig_angles.append(phi)
    if i == len(chain) - 1:
        psi = np.pi
    else:
        n_next = chain[i + 1]["N"].get_vector()
        psi = pdb.calc_dihedral(n, ca, c, n_next)
        orig_angles.append(psi)
    
orig_angles.append(np.pi)

In [33]:
len(orig_angles), len(chain) * 2

(88, 88)

## 1.2. Построим матрицу расстояний по реальным координатам $C_\beta$:

**Замечание**: Если у первого остатка нет угла $\phi_1$, то и $C_\beta$ может не быть. А что, а вдруг? 🤔

Заменим у первого остатка координату $C_\beta$ на $C_\alpha$.

**К слову**: Вы же помните, что у глицина (GLY) нет $C_\beta$? 😉

In [46]:
for i, res in enumerate(chain):
    print(i ,res)

0 <Residue GLY het=  resseq=1 icode= >
1 <Residue SER het=  resseq=2 icode= >
2 <Residue ALA het=  resseq=3 icode= >
3 <Residue ASP het=  resseq=4 icode= >
4 <Residue TYR het=  resseq=5 icode= >
5 <Residue SER het=  resseq=6 icode= >
6 <Residue SER het=  resseq=7 icode= >
7 <Residue LEU het=  resseq=8 icode= >
8 <Residue THR het=  resseq=9 icode= >
9 <Residue VAL het=  resseq=10 icode= >
10 <Residue VAL het=  resseq=11 icode= >
11 <Residue GLN het=  resseq=12 icode= >
12 <Residue LEU het=  resseq=13 icode= >
13 <Residue LYS het=  resseq=14 icode= >
14 <Residue ASP het=  resseq=15 icode= >
15 <Residue LEU het=  resseq=16 icode= >
16 <Residue LEU het=  resseq=17 icode= >
17 <Residue THR het=  resseq=18 icode= >
18 <Residue LYS het=  resseq=19 icode= >
19 <Residue ARG het=  resseq=20 icode= >
20 <Residue ASN het=  resseq=21 icode= >
21 <Residue LEU het=  resseq=22 icode= >
22 <Residue SER het=  resseq=23 icode= >
23 <Residue VAL het=  resseq=24 icode= >
24 <Residue GLY het=  resseq=25 ico

In [47]:
for i in range(len(chain)):
    print(chain[i])

<Residue GLY het=  resseq=1 icode= >
<Residue SER het=  resseq=2 icode= >
<Residue ALA het=  resseq=3 icode= >
<Residue ASP het=  resseq=4 icode= >
<Residue TYR het=  resseq=5 icode= >
<Residue SER het=  resseq=6 icode= >
<Residue SER het=  resseq=7 icode= >
<Residue LEU het=  resseq=8 icode= >
<Residue THR het=  resseq=9 icode= >
<Residue VAL het=  resseq=10 icode= >
<Residue VAL het=  resseq=11 icode= >
<Residue GLN het=  resseq=12 icode= >
<Residue LEU het=  resseq=13 icode= >
<Residue LYS het=  resseq=14 icode= >
<Residue ASP het=  resseq=15 icode= >
<Residue LEU het=  resseq=16 icode= >
<Residue LEU het=  resseq=17 icode= >
<Residue THR het=  resseq=18 icode= >
<Residue LYS het=  resseq=19 icode= >
<Residue ARG het=  resseq=20 icode= >
<Residue ASN het=  resseq=21 icode= >
<Residue LEU het=  resseq=22 icode= >
<Residue SER het=  resseq=23 icode= >
<Residue VAL het=  resseq=24 icode= >
<Residue GLY het=  resseq=25 icode= >
<Residue GLY het=  resseq=26 icode= >
<Residue LEU het=  re

In [51]:
cbs = 'B3JJ'

In [52]:
orig_matr = np.zeros((len(cbs), len(cbs)))
for i in range(len(cbs) - 1):
    for j in range(i, len(cbs)):
        orig_matr[i,j] = orig_matr[j,i] = np.linalg.norm(cbs[i] - cbs[j])

TypeError: unsupported operand type(s) for -: 'str' and 'str'

# 2. Работа с потенциалом!

In [53]:
pot = Potential(read_pickle(p4)["sequence"], p1, p2, p4, init_angles_by_dists=False)

FileNotFoundError: [Errno 2] No such file or directory: './contacts_pdb1h1j_2020_07_22_12_10_37/torsion/0/torsions/1h1j.torsions'

## 2.1. Для начала посмотрим на значения потенциала AlphaFold для оригинальных данных:

Мы получили, что весь потенциал можно сделать зависимым от $\phi, \psi$. Подставим вектор оригинальных значений $\phi, \psi$ в потенциал:

In [ ]:
pot.calc(orig_angles)

Хорош ли этот результат? 🤔

Посмотрим отдельно значения составных частей потенциала:

- $V_{distance}$:

In [ ]:
pot._calc_distance_potential(orig_angles)

- $V_{torsion}$:

In [ ]:
pot._calc_torsion_potential(orig_angles)

$\uparrow$ Какой вывод можно сделать из такого результата?

**your ideas here**: TODO

## 2.2. Минимизируем!

Выберем начальные значения углов. Пусть это будут наиболее вероятные углы с точки зрения потенциала $V_{torsion}$:

In [43]:
x0 = pot.get_initial_angles()

NameError: name 'pot' is not defined

**К слову**: а какое значение потенциала $V_{torsion}$ для $x_0$? 🤔

In [44]:
pot._calc_torsion_potential(x0)

NameError: name 'pot' is not defined

Зададим множества значений переменных. Хорошо бы поставить границы $[\pi, \pi]$, но может, стоит как-то расширить диапазон? 🤔

In [45]:
bounds = [[-1.5 * np.pi, 1.5 * np.pi]] * len(x0)

NameError: name 'x0' is not defined

Алгоритм минимизации - BFGS. Но оригинальный BFGS:

    - Не может обрабатывать ограничение множества значений переменных
    
    - Неэкономно расходует память
    
Поэтому возьмем модификацию BFGS - L-BFGS-B, которая предназначена для минимизации на ограниченных "брусах" в условиях ограниченной памяти:

In [39]:
method = "L-BFGS-B"

А еще мы можем попытаться "упростить жизнь" алгоритму за счет передачи ему функции, которая точно. вычисляет первую производную целевой функции в произвольной точке:

In [40]:
diff = pot.derivative

NameError: name 'pot' is not defined

Уфф. Запускаем!

In [41]:
from time import time

In [42]:
print("minimization started")
st_time = time()

min_res = minimize(pot.calc, 
                   x0=x0
                   bounds=bounds,
                   jac=diff,
                   method=method
                  )
print("minimization over. res = ", (min_res.success, min_res.message))
print(f"\tMinimization took {time() - st_time} sec")

SyntaxError: invalid syntax (3100620705.py, line 6)

# 3. Оценим результат 🤔

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(15,5))
im = axs[0].imshow(orig_matr)
axs[0].set_title("Матрица расстояний\nдля нативного состояния\nПотенциал = {}".format(pot.calc(orig_angles)))
cbar = plt.colorbar(im, ax=axs[0])

imp = QCPSuperimposer()
imp.set(cbs, np.array(pot._model.get_cb_coords(pot.get_initial_angles()[1:-1])[0]))
imp.run()

im = axs[1].imshow(pot._model.get_cb_dist_matrix(pot.get_initial_angles()[1:-1]))
axs[1].set_title("Матрица для $x_0$.\nПотенциал = {}\nRMSD = {}".format(pot.calc(pot.get_initial_angles()), 
                                                                        imp.get_rms()))
plt.colorbar(im, ax=axs[1])

imp = QCPSuperimposer()
imp.set(cbs, np.array(pot._model.get_cb_coords(min_res.x[1:-1])[0]))
imp.run()

im = axs[2].imshow(pot._model.get_cb_dist_matrix(min_res.x[1:-1]))
axs[2].set_title("Матрица для $x^*$ (после минимизации).\nПотенциал: {}\nRMSD = {}".format(pot.calc(min_res.x), imp.get_rms()))
plt.colorbar(im, ax=axs[2])
# fig.suptitle("Белок 1h1j. Длина: {}".format(len(chain)))
plt.show()

$\uparrow$ Что получилось? Какие можно сделать выводы? 🤔

$\rightarrow$ **your ideas here**: TODO